## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Esperance,AU,-33.8667,121.9000,66.04,70,28,18.14,scattered clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,81.68,76,33,18.48,scattered clouds
2,2,Seddon,AU,-37.8068,144.8916,79.81,36,40,11.50,scattered clouds
3,3,Bud,US,39.4470,-86.1758,31.98,58,0,6.96,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,38.86,93,75,16.11,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,81.68,76,33,18.48,scattered clouds
2,2,Seddon,AU,-37.8068,144.8916,79.81,36,40,11.50,scattered clouds
13,13,Banda Aceh,ID,5.5577,95.3222,79.86,77,98,0.40,heavy intensity rain
25,25,Rikitea,PF,-23.1203,-134.9692,76.69,72,0,18.05,clear sky
27,27,Bandarbeyla,SO,9.4942,50.8122,76.62,77,24,16.24,few clouds
34,34,Kapaa,US,22.0752,-159.3190,80.58,69,20,18.41,light rain
36,36,Carnarvon,AU,-24.8667,113.6333,84.27,58,0,20.71,clear sky
43,43,Savannah Bight,HN,16.4500,-85.8500,78.87,77,66,7.92,broken clouds
57,57,Sri Aman,MY,1.2376,111.4621,82.45,78,20,4.61,few clouds
61,61,Pangkalanbuun,ID,-2.6833,111.6167,82.27,76,100,3.13,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                170
City                   170
Country                170
Lat                    170
Lng                    170
Max Temp               170
Humidity               170
Cloudiness             170
Wind Speed             170
Current Description    170
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,81.68,76,33,18.48,scattered clouds
2,2,Seddon,AU,-37.8068,144.8916,79.81,36,40,11.50,scattered clouds
13,13,Banda Aceh,ID,5.5577,95.3222,79.86,77,98,0.40,heavy intensity rain
25,25,Rikitea,PF,-23.1203,-134.9692,76.69,72,0,18.05,clear sky
27,27,Bandarbeyla,SO,9.4942,50.8122,76.62,77,24,16.24,few clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.68,scattered clouds,-0.6000,73.0833,
2,Seddon,AU,79.81,scattered clouds,-37.8068,144.8916,
13,Banda Aceh,ID,79.86,heavy intensity rain,5.5577,95.3222,
25,Rikitea,PF,76.69,clear sky,-23.1203,-134.9692,
27,Bandarbeyla,SO,76.62,few clouds,9.4942,50.8122,
34,Kapaa,US,80.58,light rain,22.0752,-159.3190,
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,
43,Savannah Bight,HN,78.87,broken clouds,16.4500,-85.8500,
57,Sri Aman,MY,82.45,few clouds,1.2376,111.4621,
61,Pangkalanbuun,ID,82.27,overcast clouds,-2.6833,111.6167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(50)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.68,scattered clouds,-0.6000,73.0833,Scoop Guest House
2,Seddon,AU,79.81,scattered clouds,-37.8068,144.8916,Footscray Motor Inn
13,Banda Aceh,ID,79.86,heavy intensity rain,5.5577,95.3222,Sulthan Hotel International
25,Rikitea,PF,76.69,clear sky,-23.1203,-134.9692,People ThankYou
27,Bandarbeyla,SO,76.62,few clouds,9.4942,50.8122,JABIR HOTEL
34,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
43,Savannah Bight,HN,78.87,broken clouds,16.4500,-85.8500,Villa on Dunbar Rock
57,Sri Aman,MY,82.45,few clouds,1.2376,111.4621,Seri Simanggang Hotel
61,Pangkalanbuun,ID,82.27,overcast clouds,-2.6833,111.6167,Grand Kecubung Hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.68,scattered clouds,-0.6000,73.0833,Scoop Guest House
2,Seddon,AU,79.81,scattered clouds,-37.8068,144.8916,Footscray Motor Inn
13,Banda Aceh,ID,79.86,heavy intensity rain,5.5577,95.3222,Sulthan Hotel International
25,Rikitea,PF,76.69,clear sky,-23.1203,-134.9692,People ThankYou
27,Bandarbeyla,SO,76.62,few clouds,9.4942,50.8122,JABIR HOTEL
34,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
43,Savannah Bight,HN,78.87,broken clouds,16.4500,-85.8500,Villa on Dunbar Rock
57,Sri Aman,MY,82.45,few clouds,1.2376,111.4621,Seri Simanggang Hotel
61,Pangkalanbuun,ID,82.27,overcast clouds,-2.6833,111.6167,Grand Kecubung Hotel


In [11]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(50)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,81.68,scattered clouds,-0.6000,73.0833,Scoop Guest House
2,Seddon,AU,79.81,scattered clouds,-37.8068,144.8916,Footscray Motor Inn
13,Banda Aceh,ID,79.86,heavy intensity rain,5.5577,95.3222,Sulthan Hotel International
25,Rikitea,PF,76.69,clear sky,-23.1203,-134.9692,People ThankYou
27,Bandarbeyla,SO,76.62,few clouds,9.4942,50.8122,JABIR HOTEL
34,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
36,Carnarvon,AU,84.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
43,Savannah Bight,HN,78.87,broken clouds,16.4500,-85.8500,Villa on Dunbar Rock
57,Sri Aman,MY,82.45,few clouds,1.2376,111.4621,Seri Simanggang Hotel
61,Pangkalanbuun,ID,82.27,overcast clouds,-2.6833,111.6167,Grand Kecubung Hotel


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))